In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv

load_dotenv()



True

In [19]:
#make a call to this api https://www.shiftadmin.com/vjgh/org_scheduled_shifts with a body json of start_date and end_date in the format YYYY-MM-DD
#we need to use basic auth with the username and password stored in environment variables SHIFTADMIN_USER and SHIFTADMIN_PASS
import os
SHIFTADMIN_USER = os.getenv("SHIFTADMIN_USER")
SHIFTADMIN_PASS = os.getenv("SHIFTADMIN_PASS")
def fetch_shifts(start_date, end_date):
    url = "https://www.shiftadmin.com/vjgh/org_scheduled_shifts"
    body = {
        "start_date": start_date,
        "end_date": end_date
    }
    response = requests.post(url, json=body, auth=(SHIFTADMIN_USER, SHIFTADMIN_PASS))
    response.raise_for_status()
    return response.json()  

#fetch todays shifts
today = datetime.now().date()
yesterday = today - timedelta(days=1)
tomorrow = today + timedelta(days=1)
shifts = fetch_shifts(str(yesterday), str(tomorrow))
shifts_df = pd.DataFrame(shifts)
shifts_df

,scheduled_shift_id,group_id,user_id,employee_id,npi,first_name,last_name,facility_id,facility_ext_id,facility_name,...,shift_short_name,shift_start,shift_end,shift_hours,work_start,work_end,work_hours,count_as_shift,is_night,is_weekend
0,82676,1,16,None,None,Jonathan,Cooperman,1,None,Jewish General Hospital,...,W1,11/9/2025 08:00,11/9/2025 16:00,8.00,11/9/2025 08:00,11/9/2025 16:00,8.00,0,0,1
1,82677,1,68,None,None,Paola,Moresoli,1,None,Jewish General Hospital,...,X1,11/9/2025 08:00,11/9/2025 17:00,9.00,11/9/2025 08:00,11/9/2025 17:00,9.00,1,0,1
2,82678,1,33,None,None,Eli,Segal,1,None,Jewish General Hospital,...,X3,11/9/2025 08:00,11/9/2025 17:00,9.00,11/9/2025 08:00,11/9/2025 17:00,9.00,1,0,1
3,82680,1,37,None,None,Mitchell,Stendel,1,None,Jewish General Hospital,...,X4,11/9/2025 08:00,11/9/2025 17:00,9.00,11/9/2025 08:00,11/9/2025 17:00,9.00,1,0,1
4,82681,1,18,None,None,Michael,Engo,1,None,Jewish General Hospital,...,X2,11/9/2025 08:00,11/9/2025 17:00,9.00,11/9/2025 08:00,11/9/2025 17:00,9.00,1,0,1
5,82689,1,22,None,None,Alex,Guttman,1,None,Jewish General Hospital,...,WOC1,11/9/2025 08:00,11/10/2025 01:00,17.00,11/9/2025 08:00,11/10/2025 01:00,17.00,0,0,1
6,82679,1,16,None,None,Jonathan,Cooperman,1,None,Jewish General Hospital,...,X5,11/9/2025 08:00,11/9/2025 17:00,9.00,11/9/2025 08:00,11/9/2025 17:00,9.00,1,0,1
7,82682,1,10,None,None,Haran,Balendra,1,None,Jewish General Hospital,...,Y1,11/9/2025 16:00,11/10/2025 00:00,8.00,11/9/2025 16:00,11/10/2025 00:00,8.00,1,0,1
8,82683,1,8,None,None,Rafael,Aroutiunian,1,None,Jewish General Hospital,...,Y3,11/9/2025 16:00,11/10/2025 00:00,8.00,11/9/2025 16:00,11/10/2025 00:00,8.00,1,0,1
9,82685,1,35,None,None,Jonathan,Simons,1,None,Jewish General Hospital,...,Y4,11/9/2025 16:00,11/10/2025 00:00,8.00,11/9/2025 16:00,11/10/2025 00:00,8.00,1,0,1


In [20]:
#now make an hourly time series dataframe where the first column is called "ds" and is the datetime of each hour in the day, and then there is a column for each shift_short_name, and the value of each row is the user_id of the user assigned to that shift at that hour, or None if no user is assigned
time_index = pd.date_range(start=today, end=today + timedelta(days=2), freq='h')[:-1]
hourly_shifts_df = pd.DataFrame(index=time_index)
hourly_shifts_df.index.name = 'ds'
for _, shift in shifts_df.iterrows():
    # print(shift)
    # shift_start = pd.to_datetime(shift['shift_start'])
    # round shift start up to the next hour so e.g. 23:45 becomes 00:00
    shift_start = pd.to_datetime(shift['shift_start']).ceil('h')
    shift_end = pd.to_datetime(shift['shift_end']).ceil('h')
    shift_hours = pd.date_range(start=shift_start, end=shift_end, freq='h')[:-1]
    for hour in shift_hours:
        if hour in hourly_shifts_df.index:
            hourly_shifts_df.at[hour, shift['shift_short_name']] = shift['user_id']
hourly_shifts_df.reset_index(inplace=True)
hourly_shifts_df


,ds,WOC1,Z1,Z2,D1,R1,P1,D2,OC1,V1,...,B1,L1,E1,R2,P2,E2,A2,B2,N1,N2
0,2025-11-10 00:00:00,22.0,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-11-10 01:00:00,NaN,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-11-10 02:00:00,NaN,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-11-10 03:00:00,NaN,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-11-10 04:00:00,NaN,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2025-11-10 05:00:00,NaN,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2025-11-10 06:00:00,NaN,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2025-11-10 07:00:00,NaN,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2025-11-10 08:00:00,NaN,NaN,NaN,72.0,73.0,16.0,34.0,38.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2025-11-10 09:00:00,NaN,NaN,NaN,72.0,73.0,16.0,34.0,38.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#fetch the shifts starting on january 1 2021 until today, one api call per week, with 5 second delay between api calls to avoid rate limiting
all_shifts = []
start_date = datetime(2021, 1, 1).date()
end_date = today
current_start_date = start_date
while current_start_date < end_date:
    current_end_date = min(current_start_date + timedelta(days=7), end_date)
    print(f"Fetching shifts from {current_start_date} to {current_end_date}")
    shifts = fetch_shifts(str(current_start_date), str(current_end_date))
    all_shifts.extend(shifts)
    current_start_date = current_end_date + timedelta(days=1)
    import time
    time.sleep(5)  # delay to avoid rate limiting
all_shifts_df = pd.DataFrame(all_shifts)    
all_shifts_df.to_csv("all_shifts.csv", index=False)
all_shifts_df

Fetching shifts from 2021-01-01 to 2021-01-08
Fetching shifts from 2021-01-09 to 2021-01-16
Fetching shifts from 2021-01-17 to 2021-01-24
Fetching shifts from 2021-01-25 to 2021-02-01
Fetching shifts from 2021-02-02 to 2021-02-09
Fetching shifts from 2021-02-10 to 2021-02-17
Fetching shifts from 2021-02-18 to 2021-02-25
Fetching shifts from 2021-02-26 to 2021-03-05
Fetching shifts from 2021-03-06 to 2021-03-13
Fetching shifts from 2021-03-14 to 2021-03-21
Fetching shifts from 2021-03-22 to 2021-03-29
Fetching shifts from 2021-03-30 to 2021-04-06
Fetching shifts from 2021-04-07 to 2021-04-14
Fetching shifts from 2021-04-15 to 2021-04-22
Fetching shifts from 2021-04-23 to 2021-04-30
Fetching shifts from 2021-05-01 to 2021-05-08
Fetching shifts from 2021-05-09 to 2021-05-16
Fetching shifts from 2021-05-17 to 2021-05-24
Fetching shifts from 2021-05-25 to 2021-06-01
Fetching shifts from 2021-06-02 to 2021-06-09
Fetching shifts from 2021-06-10 to 2021-06-17
Fetching shifts from 2021-06-18 to